**Autor:R.** Hurtado                  

**Materia:** Procesamiento de Datos 
                            
**Topico:** Introduccion a PySpark en el Ambiente Databricks

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df00 = spark.read.table("stroke_pyspark_2_csv")
df00.columns

Out[51]: ['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke']

In [0]:
df00.show(10)


+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

In [0]:
df01 = df00.withColumn("age", df00.age.cast("int"))
df01 = df00.withColumn("bmi", df00.bmi.cast("double"))
df01.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [0]:
df01.show(10)


+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21|null|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

REVISION DE DATOS NULOS

In [0]:
df01.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df01.columns]).show()

+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
| id|gender|age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|bmi|smoking_status|stroke|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
|  0|     0|  0|           0|            0|           0|        0|             0|                0|201|             0|     0|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+



In [0]:
### promedio de IMC por Estrato  de Edades y female

avg10F = df01.where((col('gender' == lit('Female')) & (col('age') < 10))).select(mean(col('bmi'))).collect()
avg20F = df01.where((col('gender') == lit('Female')) & (col('age') > 10) & (col('age') < 20)).select(mean(col('bmi'))).collect()
avg30F = df01.where((col('gender') == lit('Female')) & (col('age') > 20) & (col('age') < 30)).select(mean(col('bmi'))).collect()
avg40F = df01.where((col('gender') == lit('Female')) & (col('age') > 30) & (col('age') < 40)).select(mean(col('bmi'))).collect()
avg50F = df01.where((col('gender') == lit('Female')) & (col('age') > 40) & (col('age') < 50)).select(mean(col('bmi'))).collect()
avg60F = df01.where((col('gender') == lit('Female')) & (col('age') > 50) & (col('age') < 60)).select(mean(col('bmi'))).collect()
avg70F = df01.where((col('gender') == lit('Female')) & (col('age') > 60) & (col('age') < 70)).select(mean(col('bmi'))).collect()
avg80F = df01.where((col('gender') == lit('Female')) & (col('age') > 70) & (col('age') < 80)).select(mean(col('bmi'))).collect()
avg90F = df01.where((col('gender') == lit('Female')) & (col('age') > 80) & (col('age') < 90)).select(mean(col('bmi'))).collect()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-1850934813616288>:3
      1 ### promedio de IMC por Estrato  de Edades y female
----> 3 avg10F = df01.where((col('gender' == lit('Female')) & (col('age') < 10))).select(mean(col('bmi'))).collect()
      4 avg20F = df01.where((col('gender') == lit('Female')) & (col('age') > 10) & (col('age') < 20)).select(mean(col('bmi'))).collect()
      5 avg30F = df01.where((col('gender') == lit('Female')) & (col('age') > 20) & (col('age') < 30)).select(mean(col('bmi'))).collect()

File /databricks/spark/python/pyspark/sql/utils.py:164, in try_remote_functions.<locals>.wrapped(*args, **kwargs)
    162     return getattr(functions, f.__name__)(*args, **kwargs)
    163 else:
--> 164     return f(*args, **kwargs)

File /databricks/spark/python/pyspark/sql/functions.py:192, in col(col)
    165 @try_remote_functions
    166 def col(col: str) 

In [0]:
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 10) avg10F[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 20) avg20F[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 30) avg30F[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 40) avg40F[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 50) avg50F[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 60) avg60F[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 70) avg70F[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 80) avg80F[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Female') & (df01['bmi'].isNull()) & (df01['age'] < 90) avg90F[0][0]).otherwise(df01['bmi']))


In [0]:
### promedio de bmi por Estrato  de Edades y Male

avg10M = df02.where((col('gender' == lit('Male')) & (col('age') < 10))).select(mean(col('bmi'))).collect()
avg20M = df02.where((col('gender') == lit('Male')) & (col('age') > 10) & (col('age') < 20)).select(mean(col('bmi'))).collect()
avg30M = df02.where((col('gender') == lit('Male')) & (col('age') > 20) & (col('age') < 30)).select(mean(col('bmi'))).collect()
avg40M = df02.where((col('gender') == lit('Male')) & (col('age') > 30) & (col('age') < 40)).select(mean(col('bmi'))).collect()
avg50M = df02.where((col('gender') == lit('Male')) & (col('age') > 40) & (col('age') < 50)).select(mean(col('bmi'))).collect()
avg60M = df02.where((col('gender') == lit('Male')) & (col('age') > 50) & (col('age') < 60)).select(mean(col('bmi'))).collect()
avg70M = df02.where((col('gender') == lit('Male')) & (col('age') > 60) & (col('age') < 70)).select(mean(col('bmi'))).collect()
avg80M = df02.where((col('gender') == lit('Male')) & (col('age') > 70) & (col('age') < 80)).select(mean(col('bmi'))).collect()

In [0]:
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 10) avg10M[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 20) avg20M[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 30) avg30M[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 40) avg40M[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 50) avg50M[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 60) avg60M[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 70) avg70M[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 80) avg80M[0][0]).otherwise(df01['bmi']))
df02 = df01.withColumn("bmi", when((df01['gender'] == 'Male') & (df01['bmi'].isNull()) & (df01['age'] < 90) avg90M[0][0]).otherwise(df01['bmi']))

In [0]:
df02.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df03.columns]).show()

In [0]:
rangos = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90)]

def calcular_promedio_imc(df, gender, rangos):
    promedios = {}
    for star, end in rangos:
        key = f'avg{end}{gender[0]}'
        avg = df.where((col('gender') == lit('genero')) & (col('age') >= start) & (col('age') < end)).select 
        (mean(col('bmi'))).collect
        promedios[key] = avg

        return promedios
    resultadosF = calcular_promedio_imc(df01, 'Female', rangos)
    print(resultadosF)
    resultadosM = calcular_promedio_imc(df01, 'Male', rangos)
    print(resultadosF)

In [0]:
##grafica edades

clases, cantidad = df04.groupby('age').count().select('count').rdd.flatMap(lambda x: x).histogram(20)
plt.hist8clases[:-1], bin = clases, weight = cantidad
plt.ylabel('cantidad')
plt.xlabel('edad')
plt.title('histograma por edades')
plt.legend(['age'], loc='upper right')
plt.show()

In [0]:
##genero
genero = df02.groupby('gender').count().select('count').rdd.flatMap(lambda x: x).collect()
#plt.hist8clases[:-1], bin = clases, weight = cantidad
categoria = ['mujeres', 'hombres']
grafica = plt.bar(categoria, genero)
grafica[0].set_color('r')
plt.ylabel('cantidad')
plt.xlabel('genero')
plt.title('grafica de barras por genero')
#plt.legend(['gender'], loc='upper right')
plt.show()

In [0]:
##enfermedades cardiacas
enfCardiaca = df02.groupby('heart_disease').count().select('count').rdd.flatMap(lambda x: x).collect()
#plt.hist8clases[:-1], bin = clases, weight = cantidad
categoria = ['si', 'no']
grafica = plt.bar(categoria, enfCardiaca)
grafica[0].set_color('r')
plt.ylabel('cantidad')
plt.xlabel('Enfermedad Cardiaca')
plt.title('grafica de barras por Enfermedad Cardiaca')
#plt.legend(['gender'], loc='upper right')
plt.show()